In [1]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from all_functions import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from aeon.transformations.detrend import STLTransformer
import ast
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error, make_scorer
import os
from sklearn.linear_model import Ridge, RidgeCV
from datetime import datetime
from distutils.util import strtobool

warnings.filterwarnings("ignore")
%matplotlib inline
def convert_to_list(series_str):
    return eval(series_str)

def get_preds_hybrid(path, test_date, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    filtered_df = df[df['test_range'] == test_date]
    columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
    values_list = columns_p1_to_p12.values.flatten().tolist()     
    results = pd.Series(values_list, index=start_index)
    return results

def generate_index(start_date, end_date):
    end_date_dt = pd.to_datetime(end_date)
    
    start_date_dt = pd.to_datetime(start_date)
    
    index = pd.period_range(start=start_date_dt, end=end_date_dt, freq='M')

    return index

/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/rapids_dask_dependency/dask_loader.py:36: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  mod = importlib.import_module(spec.name)


In [12]:
# models = ["ETS","catboost", "arima", "rf"]
# transformations = ["normal", "deseasonal"]
from dtaidistance import dtw
from tslearn.barycenters import \
    euclidean_barycenter, \
    dtw_barycenter_averaging, \
    dtw_barycenter_averaging_subgradient, \
    softdtw_barycenter

from aeon.clustering.averaging import elastic_barycenter_average
dirs = [
    '../datasets/venda/mensal/uf/gasolinac/',
    '../datasets/venda/mensal/uf/etanolhidratado/',
    '../datasets/venda/mensal/uf/glp/',
    '../datasets/venda/mensal/uf/oleodiesel/',
]
horizon = 12
window = 12
model_transformation = [
                        "GASF_ridge_deseasonal",
                        "GADF_ridge_deseasonal",
                        "STFT_ridge_deseasonal",
                        "MTF_ridge_deseasonal",
                        "RP_ridge_deseasonal",
                        "FIRTS_ridge_deseasonal",
                        "DWT_ridge_deseasonal",
                        "SWT_ridge_deseasonal",
                        "CWT_ridge_deseasonal",
                        "ETS_normal",
                        "WPT_ridge_deseasonal",
                        "catboost_deseasonal",
                        "ridge_deseasonal",
                        "arima_deseasonal"
                        ]
dates = [
        # '1993-03_1994-02',
        # '1994-03_1995-02', '1995-03_1996-02', '1996-03_1997-02', '1997-03_1998-02', '1998-03_1999-02',
        # '1999-03_2000-02', '2000-03_2001-02', '2001-03_2002-02', '2002-03_2003-02', '2003-03_2004-02',
        # '2004-03_2005-02', '2005-03_2006-02', '2006-03_2007-02', '2007-03_2008-02', '2008-03_2009-02',
        # '2009-03_2010-02', '2010-03_2011-02', '2011-03_2012-02', '2012-03_2013-02', '2013-03_2014-02',
        # '2014-03_2015-02', '2015-03_2016-02', '2016-03_2017-02', '2017-03_2018-02', '2018-03_2019-02',
        '2019-03_2020-02', '2020-03_2021-02', '2021-03_2022-02', '2022-03_2023-02', '2023-03_2024-02',
         ]

produto = "gasolinac"
estado = "SP"

combination_format = "twe"
for directory in dirs:
    all_rmse = []
    all_pocid = []
    all_mape = []
    all_pbe = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            uf = file.split("_")[1].upper()
            derivado = file.split("_")[2].split(".")[0]
            
            # if uf == estado and produto == derivado:
            full_path = os.path.join(directory, file)
            series = read_series(full_path)
            
            all_preds_dates = []
            all_test = []


            for date in dates:
                start_date, end_date = date.split('_')
                test_index = generate_index(start_date, end_date)
                test_real = get_test_real(series, start_date, end_date)
                row_data = []
                all_preds_year = []

                preds_barycenter = []
                preds_elastic = []
                
                for mt in model_transformation:
                    model, transform = mt.rsplit("_", 1)
                    
                    serie_pred = get_preds_hybrid(f'./paper_roma/{model}/{derivado}/{transform}/transform_{uf}.csv', date, test_index)
                    # print(serie_pred)
                    all_preds_year.append(serie_pred.tolist())

                    preds_barycenter.append(serie_pred.tolist())
                    
                    preds_elastic.append(np.array([[serie_pred.tolist()]]))
                    # preds_elastic.append(np.array([[serie_pred.values]]))
                
                all_test.extend(test_real)
                
                # mean_preds_year = np.mean(np.array(all_preds_year), axis=0)
                # mean_preds_year = np.median(np.array(all_preds_year), axis=0)

                if combination_format == "softdtw":
                    #softdtw barycenter
                    softdtw_preds = softdtw_barycenter(preds_barycenter, max_iter=35, gamma=0.01)
                    mean_preds_year = softdtw_preds.flatten().tolist()
                elif combination_format == "dtw_subgradient":
                    #dtw avg barycenter subradient
                    preds_dtw_subgradient = dtw_barycenter_averaging_subgradient(preds_barycenter, max_iter=35)
                    mean_preds_year = preds_dtw_subgradient.flatten().tolist()


                elif combination_format == "dtw":
                    #dtw barycenter averaging
                    preds_dtw_avg = dtw_barycenter_averaging(preds_barycenter, max_iter=35)
                    mean_preds_year = preds_dtw_avg.flatten().tolist()
                
                elif combination_format == "twe":
                    #elastic barycenter average
                    capt = np.vstack(preds_elastic)
                    elastic = elastic_barycenter_average(capt, distance="twe", reach=15),
                    mean_preds_year = elastic[0].tolist()[0]

                all_preds_dates.extend(mean_preds_year)

            rmse_result = rmse(all_test, all_preds_dates)
            mape_result = mape(all_test, all_preds_dates)
            pocid_result = pocid(all_test, all_preds_dates)
            pbe_result = pbe(pd.Series(all_test), pd.Series(all_preds_dates))
            mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

            all_pocid.append(pocid_result)
            all_pbe.append(pbe_result)
            all_rmse.append(rmse_result)
            all_mape.append(mape_result)
        
    

    if len(all_rmse) > 0:
        media_rmse = sum(all_rmse) / len(all_rmse)
        media_mape = sum(all_mape) / len(all_mape)
        media_pocid = sum(all_pocid) / len(all_pocid)
        media_pbe = sum(all_pbe) / len(all_pbe)

        representacao = combination_format

        data = {
            "representacao": [representacao],
            "RMSE": [media_rmse],
            "MAPE": [media_mape],
            "POCID": [media_pocid],
            "PBE": [media_pbe],
            "derivado": [derivado]
        }

        df = pd.DataFrame(data)

        csv_file = f'{derivado}.csv'

        df.to_csv(csv_file, mode='a', header=not pd.io.common.file_exists(csv_file), index=False)
